![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


In [1]:
-- Step 1: Identify top performing industries in 2019, 2020, and 2021
WITH top_industries AS (
    SELECT i.industry, 
           COUNT(*) AS num_unicorns
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) in (2019, 2020, 2021)
    GROUP BY i.industry
    ORDER BY num_unicorns DESC
    LIMIT 3
)
-- Display the top industries
SELECT * 
FROM top_industries;

,industry,num_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


,industry,num_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


In [3]:
-- Step 2: Gather yearly rankings data for top industries and display final result
WITH top_industries AS
(
    SELECT i.industry, 
        COUNT(i.*)
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    WHERE EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
    GROUP BY industry
    ORDER BY count DESC
    LIMIT 3
),

yearly_rankings AS 
(
    SELECT COUNT(i.*) AS num_unicorns,
        i.industry,
        EXTRACT(year FROM d.date_joined) AS year,
        AVG(f.valuation) AS average_valuation
    FROM industries AS i
    INNER JOIN dates AS d
        ON i.company_id = d.company_id
    INNER JOIN funding AS f
        ON d.company_id = f.company_id
    GROUP BY industry, year
)

SELECT industry,
    year,
    num_unicorns,
    ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019', '2020', '2021')
    AND industry in (SELECT industry
                    FROM top_industries)
GROUP BY industry, num_unicorns, year
ORDER BY year DESC, num_unicorns DESC

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58


![Dashboard 1 (2)](Dashboard%201%20(2).png)


# **Conclusion**
## 1. The number of unicorns in all three industries significantly increased in 2021.
## 2. The E-commerce & Direct-to-Consumer and Internet Software & Services industries reached their highest average valuations in 2020, with a decrease in 2021.
## 3. The Fintech industry had the highest average valuation in 2019, with a subsequent decline each year.

# Next Step：

## **I have completed identifying the three industries with the highest average valuations for unicorn companies. However, I would like to further compare these three industries with other industries using a map visualization.**

In [4]:
-- Compare top industries with other industries
WITH industry_comparison AS (
    SELECT 
        i.industry,
        EXTRACT(YEAR FROM d.date_joined) AS year,
        c.country,
        c.continent,
        COUNT(d.company_id) AS num_unicorns,
        AVG(f.valuation) AS average_valuation
    FROM industries i
    JOIN dates d ON i.company_id = d.company_id
    JOIN funding f ON i.company_id = f.company_id
    JOIN companies c ON i.company_id = c.company_id
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry, EXTRACT(YEAR FROM d.date_joined), c.country, c.continent
)
SELECT 
    industry,
    year,
    country,
    continent,
    num_unicorns,
    ROUND(average_valuation / 1e9, 2) AS average_valuation_billions
FROM industry_comparison
ORDER BY year DESC, num_unicorns DESC;

,industry,year,country,continent,num_unicorns,average_valuation_billions
0,Internet software & services,2021,United States,North America,86,2.20
1,Fintech,2021,United States,North America,74,2.65
2,Health,2021,United States,North America,29,2.17
3,Artificial intelligence,2021,United States,North America,21,1.52
4,Cybersecurity,2021,United States,North America,21,2.24
...,...,...,...,...,...,...
212,"Supply chain, logistics, & delivery",2019,Brazil,South America,1,2.00
213,Fintech,2019,Australia,Oceania,1,6.00
214,Fintech,2019,Brazil,South America,1,1.00
215,Fintech,2019,India,Asia,1,2.00


# **TOP 3 Industries**

![3 Top Industries](3%20Top%20Industries.png)


# **Other Industries**

![Other Industries](Other%20Industries.png)


# **North America:**

## **Top 3 Industries (E-commerce, Fintech, Internet Software & Services):**

- In 2021, the number of unicorns in these industries significantly increased, showing North America's strong tech innovation and market appeal.

## **Other Industries:**

- Unicorns in AI, cybersecurity, and health also saw a big rise, demonstrating broad growth in the tech sector.

# **Asia (Especially China and India):**

## **Top 3 Industries:**

- China and India saw a notable rise in unicorns, especially in e-commerce, fintech, and internet software & services, showcasing the rapid development of their digital economies.

## **Other Industries:**

- Supply chain logistics, AI, and health also experienced significant growth, highlighting the diverse tech development in Asia.

# **Europe:**

## **Top 3 Industries:**

- Europe had an increase in unicorn numbers, but the growth was smaller compared to North America and Asia, showing potential but at a slower pace.

## **Other Industries:**

- The increase in unicorn numbers was also minimal in other industries, reflecting a regional concentration of global tech innovation and investment.

# **Other Regions (South America, Africa, Oceania):**
## **Top 3 Industries:**

- These regions saw less growth in unicorn numbers for the Top 3 Industries, showing that tech innovation and market development are mainly concentrated in North America, Asia, and Europe.

## **Other Industries:**

- The increase in unicorn numbers was also minimal in other industries, reflecting a regional concentration of global tech innovation and investment.